# Join the tables

Now we are going to dive into `JOIN` queries to read data from multiple tables.

## Data
We will work with the `ecommerce.sqlite` database available at this URL:
`https://wagon-public-datasets.s3.amazonaws.com/sql_databases/ecommerce.sqlite`

Run the cell below to download the file:

In [1]:
!curl https://wagon-public-datasets.s3.amazonaws.com/sql_databases/ecommerce.sqlite > data/ecommerce.sqlite

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  9216  100  9216    0     0  49164      0 --:--:-- --:--:-- --:--:-- 49283


## Database Discovery

You already explored this databasse. Remember you can use the VS Code SQLite extension to explore the database (once you downloaded the database).

- Hit `Ctrl-Shift-P` or `Cmd-Shift-P`
- Start typing until you see `SQLite: Open Database`
- `Enter`
- Select the database file from the dropdown
- In the bottom left corner, click on `SQLITE EXPLORER`

## Setup

Pandas and sqlite3 is all we need :-)

In [2]:
import pandas as pd
from sqlite3 import connect

## Detailed Orders

👉 Write a SQL Query to get the customer contact name and the employee first name of all the orders made.

In [ ]:
# Return a list of all orders (order_id, customer.contact_name,
# employee.firstname) ordered by order_id
query_detailed_orders = """
    SELECT o.orderID,c.ContactName,e.Firstname
    FROM Orders AS o

    LEFT JOIN Customers AS c on c.CustomerID = o.CustomerID
    LEFT JOIN Employees AS e ON e.EmployeeID= o.EmployeeID

    ORDER BY orderID;
"""

In [15]:
with connect('data/ecommerce.sqlite') as conn:
    df = pd.read_sql(
        query_detailed_orders,
        con=conn
    )
df.head()

,OrderID,ContactName,FirstName
0,1,Dick Terrcotta,James
1,2,Barbara Berber,Patty
2,3,Toni Faucet,James
3,4,Barbara Berber,Lisa
4,5,Toni Faucet,Patty


When the result looks like expected, run the following cells to try and test your query.

In [16]:
from nbresult import ChallengeResult
result = ChallengeResult(
    'detailed_orders',
    query=query_detailed_orders
)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/simonhingant/.pyenv/versions/3.12.9/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /Users/simonhingant/code/simsam56/02-Data-Toolkit/05-SQL-Advanced/data-join-the-tables/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 4 items

test_detailed_orders.py::TestDetailedOrders::test_first_results PASSED   [ 25%]
test_detailed_orders.py::TestDetailedOrders::test_last_results PASSED    [ 50%]
test_detailed_orders.py::TestDetailedOrders::test_length_results PASSED  [ 75%]
test_detailed_orders.py::TestDetailedOrders::test_type_results PASSED    [100%]

============================== 4 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/detailed_orders.pickle

git commit -m 'Completed detailed_orders step'

git push origin master



## Spend per Customer

👉 Write a SQL Query to get the total amount spent per customer - in ascending order. 👌

ℹ️ We won't consider the freight charge here.

In [20]:
# Return the total amount spent per customer ordered by ascending total
# amount (to 2 decimal places)
# Example :
#     Jean   |   100
#     Marc   |   110
#     Simon  |   432
#     ...
query_spent_per_customer = """
    SELECT c.ContactName, ROUND( SUM(od.UnitPrice * od.Quantity), 2 ) AS total_spent
    FROM Customers AS c
    JOIN Orders AS o ON o.CustomerID = c.CustomerID
    JOIN OrderDetails AS od ON od.OrderID = o.OrderID
    GROUP BY c.ContactName
    ORDER BY total_spent ASC;
"""

In [21]:
with connect('data/ecommerce.sqlite') as conn:
    df = pd.read_sql(
        query_spent_per_customer,
        con=conn
    )
df.head()

,ContactName,total_spent
0,Jim Wood,1597.9
1,Dick Terrcotta,2021.7
2,Jack Diamond III,2192.6
3,Barbara Berber,5156.2
4,Toni Faucet,8700.1


When the result looks like expected, run the following cells to try and test your query.

In [22]:
from nbresult import ChallengeResult
result = ChallengeResult(
    'spent_per_customer',
    query=query_spent_per_customer
)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/simonhingant/.pyenv/versions/3.12.9/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /Users/simonhingant/code/simsam56/02-Data-Toolkit/05-SQL-Advanced/data-join-the-tables/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 4 items

test_spent_per_customer.py::TestSpentPerCustomer::test_first_result PASSED [ 25%]
test_spent_per_customer.py::TestSpentPerCustomer::test_last_result PASSED [ 50%]
test_spent_per_customer.py::TestSpentPerCustomer::test_len_resultts PASSED [ 75%]
test_spent_per_customer.py::TestSpentPerCustomer::test_type_results PASSED [100%]

============================== 4 passed in 0.02s ===============================


💯 You can commit your code:

git add tests/spent_per_customer.pickle

git commit -m 'Completed spent_per_customer step'

git push origin master



## Who sells the most:

👉 Write a SQL Query to Implement `best_employee method` to determine who's the best employee! By "best employee", we mean the one who sells the most. 👑

We expect the function to return `'FirstName', 'LastName', 6000 (the sum of all purchases)`. The order of information is irrelevant.


In [36]:
# Return the best employee! By “best employee”, we mean the one who sells the most.
# We expect: 'FirstName', 'LastName', 6000 (the sum of all purchase)).
# The order of the information is irrelevant'''
query_best_employee = """
    SELECT e.FirstName, e.LastName, ROUND(SUM(od.UnitPrice*od.quantity),2) AS total_sales
    FROM Employees AS e
    JOIN Orders AS o ON o.EmployeeID = e.EmployeeID
    JOIN  OrderDetails AS od on od.orderID=o.orderID
    GROUP BY e.EmployeeID
    ORDER BY Total_sales DESC
    LIMIT 1;
"""


In [37]:
with connect('data/ecommerce.sqlite') as conn:
    df = pd.read_sql(
        query_best_employee,
        con=conn
    )
df.head()

,FirstName,LastName,total_sales
0,Patty,Lee,7945.6


When the result looks like expected, run the following cells to try and test your query.

In [38]:
from nbresult import ChallengeResult
result = ChallengeResult(
    'best_employee',
    query=query_best_employee
)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/simonhingant/.pyenv/versions/3.12.9/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /Users/simonhingant/code/simsam56/02-Data-Toolkit/05-SQL-Advanced/data-join-the-tables/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 5 items

test_best_employee.py::TestBestEmployee::test_amount_in_results PASSED   [ 20%]
test_best_employee.py::TestBestEmployee::test_first_name_in_results PASSED [ 40%]
test_best_employee.py::TestBestEmployee::test_last_name_in_results PASSED [ 60%]
test_best_employee.py::TestBestEmployee::test_length_results PASSED      [ 80%]
test_best_employee.py::TestBestEmployee::test_type_result PASSED         [100%]

============================== 5 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/best_employee.pickle

git commit -m 'Completed best_employe

## Who doesn't buy anything?

👉 Write a SQL Query to get the amount of Orders made by each Customer. As you can guess with the title, you should also display customers with no order!

In [56]:
# Return a list with the contactName of the customer and the number of orders they made
# (contactName, number_of_orders). Order the list by ascending number of orders'''
query_orders_per_customer = """
    SELECT c.ContactName,COUNT(o.OrderID) AS NbCommande
    FROM Customers AS c
    LEFT JOIN Orders AS o ON o.CustomerID = c.CustomerID
    GROUP BY c.ContactName, c.CustomerID
    ORDER BY NbCommande ASC;

"""

In [57]:
with connect('data/ecommerce.sqlite') as conn:
    df = pd.read_sql(
        query_orders_per_customer,
        con=conn
    )
df.head()

,ContactName,NbCommande
0,Sebastien Saunier,0
1,Jack Diamond III,2
2,Dick Terrcotta,3
3,Toni Faucet,4
4,Barbara Berber,5


When the result looks like expected, run the following cells to try and test your query.

In [58]:
from nbresult import ChallengeResult
result = ChallengeResult(
    'orders_per_customer',
    query=query_orders_per_customer
)
result.write(); print(result.check())


============================= test session starts ==============================
platform darwin -- Python 3.12.9, pytest-8.3.4, pluggy-1.5.0 -- /Users/simonhingant/.pyenv/versions/3.12.9/envs/lewagon/bin/python
cachedir: .pytest_cache
rootdir: /Users/simonhingant/code/simsam56/02-Data-Toolkit/05-SQL-Advanced/data-join-the-tables/tests
plugins: anyio-4.8.0, typeguard-4.4.2
collecting ... collected 4 items

test_orders_per_customer.py::TestOrdersPerCustomer::test_first_result PASSED [ 25%]
test_orders_per_customer.py::TestOrdersPerCustomer::test_last_result PASSED [ 50%]
test_orders_per_customer.py::TestOrdersPerCustomer::test_length_results PASSED [ 75%]
test_orders_per_customer.py::TestOrdersPerCustomer::test_type_result PASSED [100%]

============================== 4 passed in 0.01s ===============================


💯 You can commit your code:

git add tests/orders_per_customer.pickle

git commit -m 'Completed orders_per_customer step'

git push origin master

